<a href="https://colab.research.google.com/github/sean-halpin/AdventOfCode/blob/master/svm_text_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("tweets_annotated.1646773409.csv")

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.tweet, df.sentiment, test_size=0.3, random_state=32)

In [11]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf.fit(X_train, y_train)

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.8633333333333333

In [15]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))
metrics.confusion_matrix(y_test, predicted)

              precision    recall  f1-score   support

    negative       0.63      0.43      0.51        63
     neutral       0.87      0.97      0.92       411
    positive       0.94      0.74      0.83       126

    accuracy                           0.86       600
   macro avg       0.81      0.71      0.75       600
weighted avg       0.86      0.86      0.85       600



array([[ 27,  35,   1],
       [  8, 398,   5],
       [  8,  25,  93]])

In [19]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [42]:
total=0
svm_equals_vader_count=0


analyzer = SentimentIntensityAnalyzer()

sentiment_strings = ["negative","neutral","positive"]
for ind in df.index:
  sentence = df['tweet'][ind]
  vs = analyzer.polarity_scores(sentence)
  sents = [vs["neg"], vs["neu"], vs["pos"]]

  svm_sentiment = df['sentiment'][ind]
  vader_sentiment = sentiment_strings[sents.index(max(sents))]
  # print("{}:{}".format(svm_sentiment, vader_sentiment))
  total+=1
  if(svm_sentiment == vader_sentiment):
    svm_equals_vader_count+=1

print("SVM and Vader agree upon; {}% of classifications".format((svm_equals_vader_count/total)*100))

SVM and Vader agree upon; 68.4% of classifications
